In [33]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

ImportError: cannot import name 'strptime' from 'datetime' (/home/daniel/anaconda3/lib/python3.7/datetime.py)

In [53]:
### Problem, this method does not exclude questions. ###
def call_pull(ticker, year, quarter,key = 'e46f1a303dafb62460de104424a00084'):
    transcript = requests.get(f'https://financialmodelingprep.com/api/v3/earning_call_transcript/{ticker}?quarter={quarter}&year={year}&apikey={key}').json()
    tran = transcript[0]['content']
    date = transcript[0]['date']
    return [tran, date]



test_tech = ['AAPL','AMZN']
test_quarters= [1,2,3,4]
test_years = [2019,2020]
call_pull('AAPL', 2020, 1)

["Operator: Good day, everyone. Welcome to the Apple Incorporated First Quarter Fiscal Year 2020 Earnings Conference Call. Today's conference is being recorded.At this time for opening remarks and introductions, I would like to turn the call over to Tejas Gala, Senior Analyst, Corporate Finance and Investor Relations. Please go ahead.\nTejas Gala: Thank you. Good afternoon, and thank you for joining us. Speaking first today is Apple's CEO, Tim Cook, and he'll be followed by CFO, Luca Maestri. After that, we'll open the call to questions from analysts.Please note that some of the information you'll hear during our discussion today will consist of forward-looking statements, including without limitation, those regarding revenue, gross margin, operating expenses, other income and expenses, taxes, capital allocation and future business outlook. Actual results or trends could differ materially from our forecast. For more information, please refer to the risk factors discussed in Apple's mos

In [59]:
def better_entire_report_pull(stock_list, year_list, quarter_list):
    report = []
    for stock in stock_list:
        for yr in year_list:
            for qtr in quarter_list:
                stonks = []
                stonks.append(stock)
#                 stonks.append(f'{yr}')
                try:
                    trans, date = call_pull(stock, yr, qtr)
                    stonks.append(trans)
                    stonks.append(date)
                except:
                    stonks.append(np.nan)
                    stonks.append(np.nan)
                report.append(stonks)
    return pd.DataFrame(report, columns = ['Ticker','Transcript', 'Date'])

In [62]:
X = better_entire_report_pull(test_tech, [2017, 2018, 2019, 2020], [1,2,3,4])

X = X.dropna(axis=0)
X

,Ticker,Transcript,Date
0,AAPL,"Operator: Good day, everyone, and welcome to t...",2017-02-01 17:00:00
1,AAPL,"Operator: Good day, everyone, and welcome to t...",2017-05-03 17:00:00
2,AAPL,"Operator: Good day, everyone, and welcome to t...",2017-08-02 17:00:00
3,AAPL,Operator: Good day everyone and welcome to thi...,2017-11-03 17:00:00
4,AAPL,Operator: Good day. And welcome to this Apple ...,2018-02-02 17:00:00
5,AAPL,"Operator: Good day everyone, and welcome to th...",2018-05-02 17:00:00
6,AAPL,Operator: Good day and welcome to the Apple In...,2018-08-01 17:00:00
7,AAPL,Operator: Good day and welcome to the Apple In...,2018-11-02 17:00:00
8,AAPL,Operator: Good day and welcome to the Apple In...,2019-01-30 17:00:00
9,AAPL,"Operator: Good day, and welcome to the Apple I...",2019-04-30 17:00:00


In [99]:
def yfin(df):
    ticker = df.Ticker
    date = df.Date[:10]
    date_ = datetime.strptime(date, '%Y-%m-%d')
    date_90 = date_+timedelta(days=90)
    try:
        data = yf.download(ticker, start=date, end=date_90, interval = '3mo')
        VGT = yf.download('VGT', start=date, end=date_90, interval = '3mo')
        
        adj = data['Adj Close'][0]
        adj_1 = data['Adj Close'][-1]
        percent_change = adj_1/adj -1
    
        VGTadj = VGT['Adj Close'][0]
        VGTadj_1 = VGT['Adj Close'][-1]
        VGTpercent_change = VGTadj_1/VGTadj -1
    except:
        percent_change = np.nan
        VGTpercent_change= np.nan

    
    return [percent_change, VGTpercent_change]

In [84]:
X['ticker_data'] = X.apply(yfin, axis=1)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

,Ticker,Transcript,Date,ticker_data
0,AAPL,"Operator: Good day, everyone, and welcome to t...",2017-02-01 17:00:00,"[0.06802837539039741, 0.04691866447556858]"
1,AAPL,"Operator: Good day, everyone, and welcome to t...",2017-05-03 17:00:00,"[0.10722097843691802, 0.034046025119629286]"
2,AAPL,"Operator: Good day, everyone, and welcome to t...",2017-08-02 17:00:00,"[nan, nan]"
3,AAPL,Operator: Good day everyone and welcome to thi...,2017-11-03 17:00:00,"[0.06767179450471561, 0.003614908105334491]"
4,AAPL,Operator: Good day. And welcome to this Apple ...,2018-02-02 17:00:00,"[0.13537369587397752, 0.07270239343305152]"
5,AAPL,"Operator: Good day everyone, and welcome to th...",2018-05-02 17:00:00,"[nan, nan]"
6,AAPL,Operator: Good day and welcome to the Apple In...,2018-08-01 17:00:00,"[nan, nan]"
7,AAPL,Operator: Good day and welcome to the Apple In...,2018-11-02 17:00:00,"[nan, nan]"
8,AAPL,Operator: Good day and welcome to the Apple In...,2019-01-30 17:00:00,"[0.06096675757245551, 0.06689151414571581]"
9,AAPL,"Operator: Good day, and welcome to the Apple I...",2019-04-30 17:00:00,"[0.08053967553571595, 0.038482175429315424]"


In [85]:
X['VGT'] = X['ticker_data'].apply(lambda x: x[1])
X['Ticker Data'] = X['ticker_data'].apply(lambda x: x[0])
X

,Ticker,Transcript,Date,ticker_data,VGT,Ticker Data
0,AAPL,"Operator: Good day, everyone, and welcome to t...",2017-02-01 17:00:00,"[0.06802837539039741, 0.04691866447556858]",0.046919,0.068028
1,AAPL,"Operator: Good day, everyone, and welcome to t...",2017-05-03 17:00:00,"[0.10722097843691802, 0.034046025119629286]",0.034046,0.107221
2,AAPL,"Operator: Good day, everyone, and welcome to t...",2017-08-02 17:00:00,"[nan, nan]",NaN,NaN
3,AAPL,Operator: Good day everyone and welcome to thi...,2017-11-03 17:00:00,"[0.06767179450471561, 0.003614908105334491]",0.003615,0.067672
4,AAPL,Operator: Good day. And welcome to this Apple ...,2018-02-02 17:00:00,"[0.13537369587397752, 0.07270239343305152]",0.072702,0.135374
5,AAPL,"Operator: Good day everyone, and welcome to th...",2018-05-02 17:00:00,"[nan, nan]",NaN,NaN
6,AAPL,Operator: Good day and welcome to the Apple In...,2018-08-01 17:00:00,"[nan, nan]",NaN,NaN
7,AAPL,Operator: Good day and welcome to the Apple In...,2018-11-02 17:00:00,"[nan, nan]",NaN,NaN
8,AAPL,Operator: Good day and welcome to the Apple In...,2019-01-30 17:00:00,"[0.06096675757245551, 0.06689151414571581]",0.066892,0.060967
9,AAPL,"Operator: Good day, and welcome to the Apple I...",2019-04-30 17:00:00,"[0.08053967553571595, 0.038482175429315424]",0.038482,0.080540


In [79]:
def yfin():
    ticker = 'AAPL'
    date = '2020-05-01'
    date_ = datetime.strptime(date, '%Y-%m-%d')
    interval = 90
    date_90 = date_+timedelta(days=interval)
    data = yf.download(ticker, start=date, end=date_90, interval = '3mo')
    VGT = yf.download('VGT', start=date, end=date_90, interval = '3mo')
    adj = data['Adj Close'][0]
    adj_1 = data['Adj Close'][-1]
    print(adj_1)
    print(type(adj_1))
#     while adj_1 == 'nan':
#         interval +=1
#         date_90 = date_+timedelta(days=interval)
#         data = yf.download(ticker, start=date, end=date_90, interval = '3mo')
#         adj_1 = data['Adj Close'][-1]
    percent_change = adj_1/adj -1
    print(data)
    return percent_change, date, date_, date_90, adj, adj_1
yfin()

[*********************100%***********************]  1 of 1 completed
nan
<class 'numpy.float64'>
               Open        High        Low       Close  Adj Close  \
Date                                                                
2020-05-01  71.5625  106.415001  71.462502  106.260002  105.44265   
2020-05-08      NaN         NaN        NaN         NaN        NaN   

                  Volume  
Date                      
2020-05-01  9.070277e+09  
2020-05-08           NaN  


(nan,
 '2020-05-01',
 datetime.datetime(2020, 5, 1, 0, 0),
 datetime.datetime(2020, 7, 30, 0, 0),
 105.4426498413086,
 nan)

In [10]:
test_tech = ['AAPL','AMZN']
test_quarters= [1,2,3,4]
test_years = [2019,2020]

def label_pull(ticker, limit=16,key = 'e46f1a303dafb62460de104424a00084'):
    label = requests.get(f'https://financialmodelingprep.com/api/v3/enterprise-values/{ticker}?period=quarter&limit={limit}&apikey=e46f1a303dafb62460de104424a00084').json()
    return pd.DataFrame(label)

def better_than_VGT(row):
    ### to be used in next function ###
    if row['Percent Change'] >= row['VGT']:
        return 1
    else:
        return 0

def full_pull(stock_list, limit=13):
    
    ### form a DataFrame that shows quarterly stock movement ###
    label = pd.DataFrame()
    skipped = []
    for stock in stock_list:
        try: 
            label = label.append(label_pull(stock, limit).iloc[::-1])
        
        except:
            skipped.append(stock)
            continue
            
    new_label = label.iloc[:,:3]
    
    ### add Percent Chane ###
#     new_label['Percent Change'] = new_label.stockPrice.pct_change()
#     new_label['Percent Change'][(new_label.date == '2017-12-30')|(new_label.date == '2017-12-31')] = np.nan

    
    ### Retrieved from https://institutional.vanguard.com/web/c1/investments/product-details/fund/0958 ###
    ### Returns from 2017-2020 all quarters ###
#     VGT = pd.DataFrame()
#     for stock in stock_list:
#         VGT = VGT.append(pd.DataFrame([.0855, .0411, .0637, .1194, -.1738, .2068, .0543, .0261, .1393, -.1315, .3185, .1202, .1389], columns = ['VGT']))
#     VGT.reset_index(inplace=True)
#     new_label.reset_index(inplace=True)
    
#     ### drop NaN values ###
#     combined_df = pd.concat([new_label, VGT], axis=1).drop('index', axis=1)
#     combined_df.dropna(inplace=True)
    
#     ### Create final column to be used as label ###
#     combined_df['Better than VGT?'] = (combined_df['Percent Change'] > combined_df['VGT']).astype(int)
    
    return new_label, skipped

y = full_pull(['AAPL'])
y

(   symbol        date  stockPrice
 12   AAPL  2017-12-30   42.307499
 11   AAPL  2018-03-31   41.669998
 10   AAPL  2018-06-30   46.277500
 9    AAPL  2018-09-29   56.435001
 8    AAPL  2018-12-29   39.057499
 7    AAPL  2019-03-30   47.487499
 6    AAPL  2019-06-29   49.480000
 5    AAPL  2019-09-28   54.705002
 4    AAPL  2019-12-28   72.449997
 3    AAPL  2020-03-28   61.935001
 2    AAPL  2020-06-27   88.407501
 1    AAPL  2020-09-26  112.279999
 0    AAPL  2020-12-26  136.690002,
 [])

In [18]:
tech = ['ACIW','ACLS','ACMR','AGYS','ALGM',
'ALRM','ALTR','AMBA','AMKR','AMSWA','APPF','APPN','APXT',
'ATOM','AVID','AVNW','AVT','AXTI','AZPN','BAND','BBSI',
'BCOV','BIGC','BL','BLKB','BMBL','BNFT','BRKS','BTRS',
'CAMP','CARG','CASA','CCMP','CCRN','CDLX','CERT','CEVA',
'CHNG','CHX','CLSK','CMTL','COMM','CPSI','CRNC','CRSR',
'CRUS','CSGS','CSOD','CVLT','DCBO','DCT','DGII','DIOD',
'DMRC','DOMO','DSGX','DSP','DSPG','EBIX','EPAY','ERII',
'EVBG','EVER','EXTR','FEYE','FORM','FROG','FSLR','FSRV',
'GDYN','GFN','GLUU','GNOG','GRPN','GTYH','HCAT','HLIT',
'HSII','HSTM','ICHR','INOV','INTZ','IPHI','JAMF','JCOM',
'KE','KELYA','KELYB','KFRC','KOPN','LASR','LCY','LECO',
'LORL','LPSN','LSCC','LTRPA','LTRPB','MANH','MARA','MCFE',
'MDRX','MGNI','MGRC','MIDD','MITK','MSGM','MSTR','MTSI',
'NATI','NCNO','NH','NTCT','NTNX','NVEC','NXGN','OMCL',
'OSIS','OSPN','PCTY','PDFS','PEGA','PFPT','PLAB','PLUS',
'PLXS','POWI','PRCH','PRFT','PRGS','PS','PUBM','QADA',
'QADB','QLYS','QMCO','RBBN','RCII','RIOT','RMBS','RP',
'RPD','RXT','SABR','SANM','SCPL','SCSC','SCWX','SGH',
'SGMS','SHLS','SITM','SLAB','SLP','SMCI','SMTC','SPSC',
'SPT','SPWR','SSPK','SSTI','SSYS','SUMO','SVMK','SWIR',
'SYKE','SYNA','TCX','TENB','TLS','TRIP','TRUE','TTEC',
'TTMI','TWOU','UCTT','UPLD','UPWK','VECO','VERI','VERX',
'VIAV','VIH','VOXX','VRNS','VRNT','VSAT','VUZI','XPER',
'ZIXI']

stonk = pd.DataFrame()
skipped = []
for stock in tech:
    try:
        stonk = stonk.append(label_pull(stock).iloc[::-1])
    except:
        skipped.append(stock)
        continue
        
stonk

,symbol,date,stockPrice,numberOfShares,marketCapitalization,minusCashAndCashEquivalents,addTotalDebt,enterpriseValue
15,ACIW,2017-03-31,21.39,116610000.0,2.494288e+09,99744000.0,696834000.0,3.091378e+09
14,ACIW,2017-06-30,22.37,117149000.0,2.620623e+09,95357000.0,692577000.0,3.217843e+09
13,ACIW,2017-09-30,22.78,118254000.0,2.693826e+09,67930000.0,692166000.0,3.318062e+09
12,ACIW,2017-12-31,22.69,118315000.0,2.684567e+09,69710000.0,685729000.0,3.300586e+09
11,ACIW,2018-03-31,23.21,115642000.0,2.684051e+09,74281000.0,679240000.0,3.289010e+09
...,...,...,...,...,...,...,...,...
4,ZIXI,2019-12-31,6.78,53199409.0,3.606920e+08,13349000.0,180100000.0,5.274430e+08
3,ZIXI,2020-03-31,4.31,53496042.0,2.305679e+08,16295000.0,185957000.0,4.002299e+08
2,ZIXI,2020-06-30,6.90,54788858.0,3.780431e+08,14057000.0,185822000.0,5.498081e+08
1,ZIXI,2020-09-30,5.84,54999114.0,3.211948e+08,23662000.0,185680000.0,4.832128e+08


In [32]:
new_stonk = stonk.iloc[:,:3]
new_stonk['Percent Change'] = new_stonk.stockPrice.pct_change()
# new_stonk['Percent Change'][(new_stonk.date == '2017-12-30')|(new_stonk.date == '2017-12-31')] = np.nan
new_stonk.symbol.value_counts()

TWOU     16
CAMP     16
LTRPB    16
AVT      16
EVBG     16
         ..
DCBO      1
SHLS      1
MSGM      1
CERT      1
PUBM      1
Name: symbol, Length: 174, dtype: int64

In [33]:
booly = (new_stonk.symbol.value_counts() == 16)
Truly = booly.index[booly==True]

In [43]:
newer_stonk = new_stonk[new_stonk.symbol.isin(Truly)]
newer_stonk.reset_index(inplace=True)
newer_stonk.symbol.unique()
newer_stonk

,index,symbol,date,stockPrice,Percent Change
0,15,ACIW,2017-03-31,21.39,NaN
1,14,ACIW,2017-06-30,22.37,0.045816
2,13,ACIW,2017-09-30,22.78,0.018328
3,12,ACIW,2017-12-31,22.69,-0.003951
4,11,ACIW,2018-03-31,23.21,0.022918
...,...,...,...,...,...
2107,4,ZIXI,2019-12-31,6.78,-0.063536
2108,3,ZIXI,2020-03-31,4.31,-0.364307
2109,2,ZIXI,2020-06-30,6.90,0.600928
2110,1,ZIXI,2020-09-30,5.84,-0.153623


In [44]:
VGT = pd.DataFrame()
for stock in newer_stonk.symbol.unique():
    VGT = VGT.append(pd.DataFrame([.1192,.0418,.0825,.0857, .0411, .0637, .1194, -.1738, .2068, .0543, .0261, .1393, -.1315, .3185, .1202, .1389], columns = ['VGT']))
VGT.reset_index(inplace=True)
VGT

,index,VGT
0,0,0.1192
1,1,0.0418
2,2,0.0825
3,3,0.0857
4,4,0.0411
...,...,...
2107,11,0.1393
2108,12,-0.1315
2109,13,0.3185
2110,14,0.1202


In [45]:
### combine and drop NaN values ###
combined_df = pd.concat([newer_stonk, VGT], axis=1).drop('index', axis=1)
combined_df.dropna(inplace=True)
    
### Create final column to be used as label ###
combined_df['Better than VGT?'] = (combined_df['Percent Change'] > combined_df['VGT']).astype(int)
y = combined_df['Better than VGT?']
combined_df

,symbol,date,stockPrice,Percent Change,VGT,Better than VGT?
1,ACIW,2017-06-30,22.37,0.045816,0.0418,1
2,ACIW,2017-09-30,22.78,0.018328,0.0825,0
3,ACIW,2017-12-31,22.69,-0.003951,0.0857,0
4,ACIW,2018-03-31,23.21,0.022918,0.0411,0
5,ACIW,2018-06-30,24.67,0.062904,0.0637,0
...,...,...,...,...,...,...
2107,ZIXI,2019-12-31,6.78,-0.063536,0.1393,0
2108,ZIXI,2020-03-31,4.31,-0.364307,-0.1315,0
2109,ZIXI,2020-06-30,6.90,0.600928,0.3185,1
2110,ZIXI,2020-09-30,5.84,-0.153623,0.1202,0


In [86]:
New_tech = ['ACIW', 'ACLS', 'AGYS', 'ALRM', 'AMBA', 'AMKR', 'AMSWA', 'APPF',
       'ATOM', 'AVID', 'AVNW', 'AVT', 'AXTI', 'AZPN', 'BBSI', 'BCOV',
       'BL', 'BLKB', 'BRKS', 'CAMP', 'CCMP', 'CCRN', 'CEVA', 'CLSK',
       'CMTL', 'COMM', 'CPSI', 'CRUS', 'CSGS', 'CSOD', 'CVLT', 'DCT',
       'DGII', 'DIOD', 'DMRC', 'DSGX', 'EBIX', 'EPAY', 'ERII', 'EVBG',
       'EXTR', 'FEYE', 'FORM', 'FSLR', 'GFN', 'GLUU', 'GRPN', 'GTYH',
       'HLIT', 'HSII', 'HSTM', 'ICHR', 'INOV', 'INTZ', 'IPHI', 'JCOM',
       'KE', 'KELYA', 'KELYB', 'KFRC', 'KOPN', 'LECO', 'LORL', 'LPSN',
       'LSCC', 'LTRPA', 'LTRPB', 'MANH', 'MARA', 'MDRX', 'MGRC', 'MIDD',
       'MITK', 'MTSI', 'NATI', 'NH', 'NTCT', 'NTNX', 'NVEC', 'NXGN',
       'OMCL', 'OSIS', 'PCTY', 'PDFS', 'PEGA', 'PFPT', 'PLAB', 'PLUS',
       'PLXS', 'POWI', 'PRFT', 'PRGS', 'QADA', 'QADB', 'QLYS', 'QMCO',
       'RBBN', 'RCII', 'RMBS', 'RP', 'RPD', 'SABR', 'SANM', 'SCSC',
       'SCWX', 'SGMS', 'SLAB', 'SLP', 'SMCI', 'SMTC', 'SPSC', 'SPWR',
       'SSYS', 'SWIR', 'SYKE', 'SYNA', 'TCX', 'TRIP', 'TRUE', 'TTEC',
       'TTMI', 'TWOU', 'UCTT', 'UPLD', 'VECO', 'VIAV', 'VOXX', 'VRNS',
       'VRNT', 'VSAT', 'VUZI', 'ZIXI']
len(New_tech)

132

In [88]:
X = better_entire_report_pull(New_tech, [2017,2018,2019,2020], [1,2,3,4])

In [97]:
X = X.dropna()
X

,Ticker,Transcript,Date
0,ACIW,"Operator: Good morning. My name is Jacquelyn, ...",2017-05-04 17:00:00
1,ACIW,"Operator: Good morning. My name is Hyde, and I...",2017-07-27 17:00:00
2,ACIW,"Operator: Ladies and gentlemen, thank you for ...",2017-11-04 17:00:00
3,ACIW,"Operator: Good morning. My name is Heidi, and ...",2018-02-24 17:00:00
4,ACIW,"Operator: Good day. My name is Elsa, and I wil...",2018-05-12 17:00:00
...,...,...,...
2006,VIAV,Operator: Good afternoon. My name is Jessa [ph...,2018-05-04 17:00:00
2007,VIAV,"Operator: Good afternoon. My name is Cheryl, a...",2018-08-15 17:00:00
2008,VIAV,"Operator: Good afternoon. My name is Chris, an...",2018-11-02 17:00:00
2009,VIAV,"Operator: Good afternoon, my name is Christine...",2019-02-06 17:00:00


In [100]:
X['ticker_data'] = X.apply(yfin, axis=1)
X

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- DCT: Data doesn't exist for startDate = 1494043200, endDate = 1501819200
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- DCT: Data doesn't exist for startDate = 1501819200, endDate = 1509595200
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- DCT: Data doesn't exist for startDate = 1509681600, endDate = 1517461200
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- DCT: Data doesn't exist for startDate = 1517547600, endDate = 1525320000

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

...

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

,Ticker,Transcript,Date,ticker_data
0,ACIW,"Operator: Good morning. My name is Jacquelyn, ...",2017-05-04 17:00:00,"[-0.01769528899706807, 0.034046025119629286]"
1,ACIW,"Operator: Good morning. My name is Hyde, and I...",2017-07-27 17:00:00,"[0.057067567948874354, 0.07697192205036196]"
2,ACIW,"Operator: Ladies and gentlemen, thank you for ...",2017-11-04 17:00:00,"[0.00895900510614478, 0.003614908105334491]"
3,ACIW,"Operator: Good morning. My name is Heidi, and ...",2018-02-24 17:00:00,"[0.03870966101205475, 0.07270239343305152]"
4,ACIW,"Operator: Good day. My name is Elsa, and I wil...",2018-05-12 17:00:00,"[0.09945819193684602, 0.09235149189471348]"
...,...,...,...,...
2006,VIAV,Operator: Good afternoon. My name is Jessa [ph...,2018-05-04 17:00:00,"[0.10671936125682957, 0.09235149189471348]"
2007,VIAV,"Operator: Good afternoon. My name is Cheryl, a...",2018-08-15 17:00:00,"[-0.12055502357676506, -0.011487246943179374]"
2008,VIAV,"Operator: Good afternoon. My name is Chris, an...",2018-11-02 17:00:00,"[0.0, nan]"
2009,VIAV,"Operator: Good afternoon, my name is Christine...",2019-02-06 17:00:00,"[-0.09398496105817977, -0.08606154482966732]"


In [ ]:
X['VGT'] = X['ticker_data'].apply(lambda x: x[1])
X['Ticker Data'] = X['ticker_data'].apply(lambda x: x[0])
X

In [104]:
X['Better than VGT?'] = (X['Ticker Data'] > X['VGT']).astype(int)
X


,Ticker,Transcript,Date,ticker_data,VGT,Ticker Data,Better than VGT?
0,ACIW,"Operator: Good morning. My name is Jacquelyn, ...",2017-05-04 17:00:00,"[-0.01769528899706807, 0.034046025119629286]",0.034046,-0.017695,0
1,ACIW,"Operator: Good morning. My name is Hyde, and I...",2017-07-27 17:00:00,"[0.057067567948874354, 0.07697192205036196]",0.076972,0.057068,0
2,ACIW,"Operator: Ladies and gentlemen, thank you for ...",2017-11-04 17:00:00,"[0.00895900510614478, 0.003614908105334491]",0.003615,0.008959,1
3,ACIW,"Operator: Good morning. My name is Heidi, and ...",2018-02-24 17:00:00,"[0.03870966101205475, 0.07270239343305152]",0.072702,0.038710,0
4,ACIW,"Operator: Good day. My name is Elsa, and I wil...",2018-05-12 17:00:00,"[0.09945819193684602, 0.09235149189471348]",0.092351,0.099458,1
...,...,...,...,...,...,...,...
2006,VIAV,Operator: Good afternoon. My name is Jessa [ph...,2018-05-04 17:00:00,"[0.10671936125682957, 0.09235149189471348]",0.092351,0.106719,1
2007,VIAV,"Operator: Good afternoon. My name is Cheryl, a...",2018-08-15 17:00:00,"[-0.12055502357676506, -0.011487246943179374]",-0.011487,-0.120555,0
2008,VIAV,"Operator: Good afternoon. My name is Chris, an...",2018-11-02 17:00:00,"[0.0, nan]",NaN,0.000000,0
2009,VIAV,"Operator: Good afternoon, my name is Christine...",2019-02-06 17:00:00,"[-0.09398496105817977, -0.08606154482966732]",-0.086062,-0.093985,0


In [106]:
y = X['Better than VGT?']
y

0       0
1       0
2       1
3       0
4       1
       ..
2006    1
2007    0
2008    0
2009    0
2010    0
Name: Better than VGT?, Length: 803, dtype: int64

In [18]:
stonk.to_csv('../data/y.csv')

In [105]:
X.to_csv('../data/X_2.csv')

In [108]:
y.to_csv('../data/y.csv')

In [77]:
df = pd.concat([X,y],axis=1)
# df = df.dropna()
df = df[df['Date'] != '2017,1 Qtr']
df

,Ticker,Date,Transcript,Better than VGT?
1,ACIW,"2017,2 Qtr","Operator: Good morning. My name is Hyde, and I...",1.0
2,ACIW,"2017,3 Qtr","Operator: Ladies and gentlemen, thank you for ...",0.0
3,ACIW,"2017,4 Qtr","Operator: Good morning. My name is Heidi, and ...",0.0
4,ACIW,"2018,1 Qtr","Operator: Good day. My name is Elsa, and I wil...",0.0
5,ACIW,"2018,2 Qtr","Operator: Good day. My name is Karnesia, and I...",0.0
...,...,...,...,...
2107,ZIXI,"2019,4 Qtr",Operator: Good afternoon. Welcome to Zix's Fou...,0.0
2108,ZIXI,"2020,1 Qtr","Operator: Good afternoon, and welcome to Zix's...",0.0
2109,ZIXI,"2020,2 Qtr",Operator: Good afternoon. Welcome to Zix Secon...,1.0
2110,ZIXI,"2020,3 Qtr",None,0.0


In [78]:
df.to_csv('../data/data.csv')